# 0. Build Team Identities, Company Employment and User Identities

Notebook to build all of the different identities assumed available when I have initiailised Type Refinery. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating. Note we have added on some system identities, such as Exchange server, and some Laptops

We will take the Type Refinery User as an Example of the Process of making a group of Complex Identity using the official [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/tree/main/extension-definition-specifications/identity-contact-information) specification, and combining it with the standard Stix approach. There are five stages:

1. Create the `User_Account` first,
2. Then the `Email_Address` objects, attaching the new User_Account.["id"]
3. Then create the `EmailContact`, `SocialMediaContact`, `ContactNumber` lists of sub-objects from the specification, attaching their User_Account.["id"] and Email_Address.["id"] as required
4. Create the [Identity Stix object](https://docs.oasis-open.org/cti/stix/v2.1/os/stix-v2.1-os.html#_wh296fiwpklp) with the [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/identity-contact-information/Identity%20Contact%20Information.docx), and joining the sub objects to the Extension object and then down to the base Identity object
5. Collect all of the objects and put them into a bundle, and save into a json

Then we will create identities fro the three dummy users that are always clicking the wrong links, but at least they are good on reporting. Finally, we will employ the four Identities with our current company, and save the complete group of objects into a bundle

Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (It also verifies the objects are created correctly, and is cool).

![](img/Evidence_Identities.png)
![](img/Evidence_Identities_DataFlows.png)

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks and Phishing Scenario Data

Also the data needed to initialise the company working on the phishing scenario

In [3]:
import json
from Utilities.local_make_general import invoke_save_options_context_block, invoke_save_incident_context_block
from Utilities.util import emulate_ports, unwind_ports, conv
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block
from Utilities.local_make_general import invoke_update_company_relations_block

path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_attributed.json"
identities_data = [
    {
        "who": "me", "email": "SCO/Email_Addr/email_addr_TR_user.json", 
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "step0/TR_Identity"
    }, 
    {
        "who": "usr1", "email": "SCO/Email_Addr/email_addr_IT_user1.json", 
        "acct": "SCO/User_Account/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "step0/Usr1"
    }, 
    {
        "who": "usr2", "email": "SCO/Email_Addr/email_addr_IT_user2.json", 
        "acct": "SCO/User_Account/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "step0/Usr2"
    }, 
    {
        "who": "usr3", "email": "SCO/Email_Addr/email_addr_IT_user3.json", 
        "acct": "SCO/User_Account/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "step0/Usr3"
    }, 
]
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "step0/employer"
}
systems_base = [
    {
        "data_path": "SDO/Identity/identity_Exchange.json",
        "results": "step0/Systems_Exchange"
    }
]
assets_base = [
    {
        "data_path": "SDO/Identity/identity_Laptop1.json",
        "results": "step0/Systems_Laptop1"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop2.json",
        "results": "step0/Systems_Laptop2"
    }
]
results_paths = {}

## B. Create the Multi-Faceted User of this Type Refinery

only perform once on initialisation, otherwise you must update the identity object

### B.1 First Select the User Data

In [4]:
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]


### B.2 Make the User Account for the TR User
Simply pass the paths for the input file and where the results should be written

In [5]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_user_acct = invoke_make_user_account_block(acct_path, results_path)
# 2. add the record to the in-session bundles and context lists
context_type = {
    "context_type": "me"
}
acct_results_obj_path = results_base + results_path + "__usr_acct.json"
acct_results_context_path = results_base + "step0/context/TR__usr_acct_context.json"
result = invoke_save_options_context_block(acct_results_obj_path, acct_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}
 result->{'context_result': ' context saved -> me'}


### B.3 Make the Email Address for the TR User

In [6]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_email_addr = invoke_make_email_addr_block(email_path, results_path, TR_user_acct)
# 2. add the record to the in-session bundles and context lists
context_type = {
    "context_type": "me"
}
email_results_obj_path = results_base + results_path + "__email.json"
email_results_context_path = results_base + "step0/context/TR__email_addr_context.json"
result = invoke_save_options_context_block(email_results_obj_path, email_results_context_path, context_type)
print(f" result->{result}")

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}
 result->{'context_result': ' context saved -> me'}


### B.4 Make the Identity for the TR User

In [7]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_ident = invoke_make_identity_block(ident_path, results_path, email_results=TR_email_addr, acct_results=TR_user_acct)
# 2. add the record to the in-session bundles and context lists
context_type = {
    "context_type": "me"
}
ident_results_obj_path = results_base + results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/TR__ident_context.json"
result = invoke_save_options_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")

type identity-><class 'dict'>, type email-><class 'dict'>, type user acct-><class 'dict'>
Step 1 >>
Step 2 >>
#>> v [{'digital_contact_type': 'work'}]
#>> email_addrs {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3', 'value': 'trusty@example.com', 'display_name': 'Trusty Jones', 'belongs_to_ref': 'user-account--83658594-537d-5c32-b9f0-137354bd9bc3'}
#>> v [{'digital_contact_type': 'work'}]
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--0c47928f-38f4-4be0-b08c-591620774c1a",
    "created": "2024-03-03T13:38:09.668Z",
    "modified": "2024-03-03T13:38:09.668Z",
    "name": "Trusty Jones",
    "description": "A Trusty Individual",
    "roles": [
        "soc",
        "support"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-exten

## C. Create the Company

create the company object

In [8]:
cwd = os.getcwd()
#
# Setup company info into paths
company_ident_path = company_base["ident"]
company_results_path = company_base["results"]
#
# MAke the Company object nd add it to the bundle_list
#
comp_ident = invoke_make_identity_block(company_ident_path, company_results_path)
# 2. add the record to the in-session bundles and context lists
context_type = {
    "context_type": "company"
}
ident_results_obj_path = results_base + company_results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/company__ident_context.json"
result = invoke_save_options_context_block(ident_results_obj_path, ident_results_context_path, context_type)
print(f" result->{result}")

type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--99a7eec5-96be-40c6-a243-57594af7644a",
    "created": "2024-03-03T13:38:09.779Z",
    "modified": "2024-03-03T13:38:09.779Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}
 result->{'context_result': ' context saved -> company'}


## D. Create the Company Users and Add them to Context

For each data record in the list, make the user account, the email-addr and identity

In [9]:
for ident in identities_data:
    results_paths = {}
    if ident["who"][:3] == "usr":
        # 1. Object Paths and Results Path
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        # 2. Options Context Type
        context_type = {
            "context_type": "users"
        }
        # 3, Context Pathways
        acct_results_obj_path = results_base + results_path + "__usr_acct.json"
        acct_results_context_path = results_base  + "step0/context/usr_" + results_path[-1:] + "__usr_acct_context.json"
        email_results_obj_path = results_base + results_path + "__email.json"
        email_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__email.json"
        ident_results_obj_path = results_base + results_path + "__ident.json"
        ident_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__ident_context.json"
        # 4. invoke the account object using method from ./Utilities/local_make_sco.py, print it and add it to context
        user_acct = invoke_make_user_account_block(acct_path, results_path)
        result = invoke_save_options_context_block(acct_results_obj_path, acct_results_context_path, context_type)
        print(f" result->{result}")
        user_email_addr = invoke_make_email_addr_block(email_path, results_path, user_acct)
        result = invoke_save_options_context_block(email_results_obj_path, email_results_context_path, context_type)
        print(f" result->{result}")
        user_ident = invoke_make_identity_block(ident_path, results_path,email_results=user_email_addr, acct_results=user_acct)
        result = invoke_save_options_context_block(ident_results_obj_path, ident_results_context_path, context_type)
        print(f" result->{result}")

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
    "user_id": "79563902",
    "account_login": "nsmith",
    "account_type": "sales,",
    "display_name": "Naive Smith"
}
 result->{'context_result': ' context saved -> users'}
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
    "value": "naive@example.com",
    "display_name": "Naive Smith",
    "belongs_to_ref": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b"
}
 result->{'context_result': ' context saved -> users'}
type identity-><class 'dict'>, type email-><class 'dict'>, type user acct-><class 'dict'>
Step 1 >>
Step 2 >>
#>> v [{'digital_contact_type': 'work'}]
#>> email_addrs {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--4722424c-7012-56b0-84d5-01d076fc547b', 'value': 'naive@example.com', 'display_name': 'Naive Smith', 'belongs_to_ref': 'user-account--597ad4d4-35ba-585

## E. Create and Save the List of IT System Identities


In [10]:
for IT_system in systems_base:
    #
    # Setup company info into paths
    system_ident_path = IT_system["data_path"]
    system_results_path = IT_system["results"]
    # 2. Options Context Type
    context_type = {
        "context_type": "systems"
    }
    #
    # MAke the Company object nd add it to the bundle_list
    #
    system_ident = invoke_make_identity_block(system_ident_path, system_results_path)
    # 2. add the record to the in-session bundles and context lists
    ident_results_obj_path = results_base + system_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/system" + system_results_path[-1:]  + "__ident_context.json"
    result = invoke_save_options_context_block(ident_results_obj_path, ident_results_context_path, context_type)
    print(f" result->{result}")

type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--16382555-d88d-434e-ac7c-5be0d127e72c",
    "created": "2024-03-03T13:38:10.307Z",
    "modified": "2024-03-03T13:38:10.307Z",
    "name": "Microsoft Exchange",
    "description": "My Local Exchange Server",
    "roles": [
        "server"
    ],
    "identity_class": "system",
    "sectors": [
        "technology"
    ],
    "external_references": [
        {
            "source_name": "Exchange Server",
            "description": "our email server",
            "url": "https://mail.echange.microsoft.com",
            "external_id": "admin"
        }
    ]
}
 result->{'context_result': ' context saved -> systems'}


## F. Create and Save the List of IT Hardware Assets


In [11]:
for i, asset in enumerate(assets_base):
    #
    # Setup company info into paths
    asset_ident_path = asset["data_path"]
    asset_results_path = asset["results"]

    # 2. Options Context Type
    context_type = {
        "context_type": "assets"
    }
    #
    # MAke the Company object nd add it to the bundle_list
    #
    asset_ident_list = invoke_make_identity_block(asset_ident_path, asset_results_path)
    # 2. add the record to the in-session bundles and context lists
    ident_results_obj_path = results_base + asset_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/hardware" + asset_results_path[-1:]  + str(i) + "__ident_context.json"
    result = invoke_save_options_context_block(ident_results_obj_path, ident_results_context_path, context_type)
    print(f" result->{result}")

type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--bc2a7996-f407-423d-8cbe-f87d852d1c40",
    "created": "2024-03-03T13:38:10.365Z",
    "modified": "2024-03-03T13:38:10.365Z",
    "name": "HP Laptop 1",
    "description": "Usr 1's Laptop",
    "roles": [
        "laptop"
    ],
    "identity_class": "asset",
    "sectors": [
        "technology"
    ],
    "external_references": [
        {
            "source_name": "HP Ultima 10",
            "description": "16GB RAM, 1TB SD, Microsoft Windows 11 Professional",
            "url": "https://mail.echange.microsoft.com",
            "external_id": "Asset--629456a"
        }
    ]
}
 result->{'context_result': ' context saved -> assets'}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",


## G. Create and Save any SRO's needed


In [12]:
#
# # Add all relations.contracted-by, employ, asset-of, and system-of
# #
stix_object_path = "../Block_Families/Context/Update_context/relationship_type.json"
results_path = results_base + "step0/_relations_made.json"
result_string = invoke_update_company_relations_block(stix_object_path, results_path)



print(f" result->{result}")

 result->{'context_result': ' context saved -> assets'}
